In [5]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

In [6]:
#Read OPENAI_API_KEY from environment variable
openai_api_key = os.environ['OPENAI_API_KEY']
embeddings = OpenAIEmbeddings(disallowed_special=())

In [7]:
#Login to ActiveLoop using API Key environment variable
active_loop_api_key = os.environ['ACTIVE_LOOP_API_KEY']
!activeloop login -t $active_loop_api_key

Successfully logged in to Activeloop.


In [8]:
import os
from langchain.document_loaders import TextLoader

root_dir = 'repo'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except:
            pass

In [9]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 1270, which is longer than the specified 1000
Created a chunk of size 1535, which is longer than the specified 1000
Created a chunk of size 1673, which is longer than the specified 1000
Created a chunk of size 1390, which is longer than the specified 1000
Created a chunk of size 1115, which is longer than the specified 1000
Created a chunk of size 1565, which is longer than the specified 1000
Created a chunk of size 1502, which is longer than the specified 1000
Created a chunk of size 1432, which is longer than the specified 1000
Created a chunk of size 1205, which is longer than the specified 1000
Created a chunk of size 1332, which is longer than the specified 1000
Created a chunk of size 2442, which is longer than the specified 1000
Created a chunk of size 1111, which is longer than the specified 1000
Created a chunk of size 3002, which is longer than the specified 1000
Created a chunk of size 1797, which is longer than the specified 1000
Created a chunk of s

In [11]:
username = os.environ['username']
db_name = os.environ['db_name']
#TODO: Better handling for retrying due to RateLimitError from OpenAI
db = DeepLake(
    dataset_path=f"hub://{username}/{db_name}",
    embedding_function=embeddings,
)
db.add_documents(texts)

Deep Lake Dataset in hub://bhagirathbhardwaj/jockmkt already exists, loading from the storage


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: OpenAI API returned an empty embedding.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-7fbpd8wLY1Ik8V4ss5aIkC6x on tokens per min. Limit: 1000000 / min. Current: 687447 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-7fbpd8wLY1Ik8V4ss5aIkC6x on tokens per min. Limit: 1000000 / min. Current: 605442 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 8.0 sec

Dataset(path='hub://bhagirathbhardwaj/jockmkt', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
 embedding  embedding  (6697, 1536)  float32   None   
    id        text      (6697, 1)      str     None   
 metadata     json      (6697, 1)      str     None   
   text       text      (6697, 1)      str     None   


['5313024c-20d6-11ee-b3c1-eed7f53b3185',
 '53130350-20d6-11ee-b3c1-eed7f53b3185',
 '53130396-20d6-11ee-b3c1-eed7f53b3185',
 '531303c8-20d6-11ee-b3c1-eed7f53b3185',
 '531303f0-20d6-11ee-b3c1-eed7f53b3185',
 '53130418-20d6-11ee-b3c1-eed7f53b3185',
 '5313044a-20d6-11ee-b3c1-eed7f53b3185',
 '531304a4-20d6-11ee-b3c1-eed7f53b3185',
 '531304e0-20d6-11ee-b3c1-eed7f53b3185',
 '53130508-20d6-11ee-b3c1-eed7f53b3185',
 '53130530-20d6-11ee-b3c1-eed7f53b3185',
 '53130558-20d6-11ee-b3c1-eed7f53b3185',
 '53130580-20d6-11ee-b3c1-eed7f53b3185',
 '531305a8-20d6-11ee-b3c1-eed7f53b3185',
 '531305d0-20d6-11ee-b3c1-eed7f53b3185',
 '531305f8-20d6-11ee-b3c1-eed7f53b3185',
 '53130620-20d6-11ee-b3c1-eed7f53b3185',
 '53130648-20d6-11ee-b3c1-eed7f53b3185',
 '53130666-20d6-11ee-b3c1-eed7f53b3185',
 '5313068e-20d6-11ee-b3c1-eed7f53b3185',
 '531306b6-20d6-11ee-b3c1-eed7f53b3185',
 '531306de-20d6-11ee-b3c1-eed7f53b3185',
 '53130706-20d6-11ee-b3c1-eed7f53b3185',
 '5313072e-20d6-11ee-b3c1-eed7f53b3185',
 '53130756-20d6-

In [12]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo")
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [18]:
# Read questions from the text file
with open('questions.txt', 'r') as file:
    questions = file.read().splitlines()

chat_history = []

for index, question in enumerate(questions):
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    #print(f"**Answer**: {result['answer']} \n")
    # Write the answer to a separate file
    answer_filename = f'answers/answer{index+1}.md'
    with open(answer_filename, 'w') as answer_file:
        answer_file.write(f"# Answer to Question {index+1}\n\n")
        answer_file.write(result["answer"])

-> **Question**: Give suggestions for using an API that looks at sportsbook odds for each player that we can make as part of our trading strategy in jockmkt_trading_bot 

-> **Question**: I want to use https://the-odds-api.com/ for creating a trading bot on jockmkt_trading_bot show a sample implementation 

